## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-19-01-10-58 +0000,nypost,AFT boss Randi Weingarten promotes her book su...,https://nypost.com/2025/09/18/us-news/randi-we...
1,2025-09-19-01-10-03 +0000,48hills,"The Engardio recall, Yimby urbanist elitism, a...",https://48hills.org/2025/09/the-engardio-recal...
2,2025-09-19-01-05-00 +0000,wsj,Trump’s Trade War Split Canada and Mexico. Now...,https://www.wsj.com/world/americas/canada-mexi...
3,2025-09-19-01-00-00 +0000,wsj,President Trump’s team is weighing a plan to s...,https://www.wsj.com/politics/policy/trump-manu...
4,2025-09-19-01-00-00 +0000,wsj,Solar-Powered Cars and Trucks Are Almost Here,https://www.wsj.com/business/energy-oil/aptera...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2336/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
8,trump,67
84,kimmel,29
86,jimmy,24
186,new,19
12,kirk,17


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
202,2025-09-18-15-10-04 +0000,nypost,Trump claims Jimmy Kimmel was taken off air fo...,https://nypost.com/2025/09/18/us-news/trump-cl...,187
10,2025-09-19-00-40-36 +0000,nypost,Jimmy Kimmel raged against Trump and his ‘MAGA...,https://nypost.com/2025/09/18/us-news/jimmy-ki...,180
63,2025-09-18-21-50-35 +0000,nyt,What Jimmy Kimmel’s Suspension Says About the ...,https://www.nytimes.com/2025/09/18/business/me...,160
122,2025-09-18-19-53-00 +0000,wsj,The Trump administration is putting the weight...,https://www.wsj.com/politics/policy/jimmy-kimm...,154
113,2025-09-18-20-13-00 +0000,wsj,The Trump administration asked the Supreme Cou...,https://www.wsj.com/politics/policy/trump-admi...,153


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
202,187,2025-09-18-15-10-04 +0000,nypost,Trump claims Jimmy Kimmel was taken off air fo...,https://nypost.com/2025/09/18/us-news/trump-cl...
113,86,2025-09-18-20-13-00 +0000,wsj,The Trump administration asked the Supreme Cou...,https://www.wsj.com/politics/policy/trump-admi...
23,61,2025-09-19-00-00-33 +0000,nypost,Billionaire Mets owner Steve Cohen emerges as ...,https://nypost.com/2025/09/18/us-news/billiona...
204,54,2025-09-18-14-58-31 +0000,nypost,Jimmy Kimmel’s ABC suspension sparks boycott c...,https://nypost.com/2025/09/18/media/jimmy-kimm...
318,47,2025-09-18-03-38-00 +0000,wsj,President Trump is frustrated with Israeli Pri...,https://www.wsj.com/world/middle-east/trump-ne...
201,46,2025-09-18-15-15-02 +0000,nypost,Trump admits Putin ‘let me down’ on Ukraine pe...,https://nypost.com/2025/09/18/us-news/trump-ad...
282,45,2025-09-18-09-30-00 +0000,wsj,Capital Account: Whatever probability you put ...,https://www.wsj.com/economy/central-banking/th...
297,43,2025-09-18-08-08-57 +0000,wsj,Stock Market Today: Dow Rises After Fed Signal...,https://www.wsj.com/livecoverage/stock-market-...
122,42,2025-09-18-19-53-00 +0000,wsj,The Trump administration is putting the weight...,https://www.wsj.com/politics/policy/jimmy-kimm...
181,40,2025-09-18-16-17-43 +0000,latimes,Teenager Celeste Rivas went missing a year ago...,https://www.latimes.com/california/story/2025-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
